In [ ]:
import datetime
from ibapi import wrapper
from ibapi import client
from ibapi import contract
from collections import defaultdict


class Wrapper(wrapper.EWrapper):
    def __init__(self):
        pass

class Client(client.EClient):
    def __init__(self, wrapper):
        client.EClient.__init__(self, wrapper)

class App(Wrapper, Client):
    
    def __init__(self):
        Wrapper.__init__(self)
        Client.__init__(self, wrapper=self)
        self.__unq_id = 0
        self.historical_data = defaultdict(list)


    def historicalData(self, reqId, bar):
        print("HistoricalData. ReqId:", reqId, "BarData.", end='\r')
        self.historical_data[reqId].append(bar)
        trade_tm_dt = datetime.datetime.strptime(
            bar.date[:-2] + '00', '%Y%m%d  %H:%M:%S'
        )

    
                 
    def historicalDataEnd(self, reqId: int, start: str, end: str):
        super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        super().disconnect()


    def get_contract(self, symbol, secType='CASH', currency='USD', exchange='IDEALPRO', futures_month=None):
        sec_contract = contract.Contract()
        sec_contract.symbol = symbol
        sec_contract.secType = secType
        sec_contract.currency = currency
        sec_contract.exchange = exchange
        return sec_contract

    def get_unique_id(self, filepath='counter.txt'):
        counter = 1
        if self.__unq_id is None:
            if not opath.exists(filepath):
                with open(filepath, 'w') as cnt_file:
                    cnt_file.write('1')
            else:
                with open(filepath, 'r') as cnt_file:
                    counter = int(cnt_file.read())
                with open(filepath, 'w') as cnt_file:
                    cnt_file.write(str(counter + 5))
        else:
            counter = self.__unq_id + 5
            self.__unq_id = counter
        return counter

    
    def get_historical_data(self, symbol, secType='CASH', exchange='IDEALPRO',
                            history_len=5, history_unit='D', bar_unit='hour', bar_length=1,
                            only_RTH=0, **kwargs):
        unq_id = self.get_unique_id()
        req_id = int(f'{unq_id}')
        self.__current_req_id = req_id
        sec_contract = self.get_contract(symbol, secType=secType, exchange=exchange)
        sym_data = self.reqHistoricalData(req_id, sec_contract, "", f"{history_len} {history_unit}",
                                          f"{bar_length} {bar_unit}", 'BID', only_RTH, 1, False, [])
        self.__proc_req_ids = [req_id]
        return req_id

In [ ]:
app = App()

In [ ]:
app.connect('127.0.0.1',  7497, clientId=0)

In [ ]:
app.get_historical_data('GBP', history_len=30)

In [ ]:
app.run()

In [ ]:
import pandas as pd


def bar_to_df(historical_data):
    data = []
    columns = ['date', 'open', 'close', 'high', 'low']
    for key in historical_data:
        for bar in historical_data[key]:
            date = datetime.datetime.strptime(bar.date, '%Y%m%d %H:%M:%S')
            data.append(
                [date, bar.open, bar.close, bar.high, bar.low]
            )
    return pd.DataFrame(data, columns=columns)
            

In [ ]:
df = bar_to_df(app.historical_data)

In [ ]:
df = df.assign(min_rolling=df.low.rolling(15).min())

In [ ]:
df = df.assign(last_min_rolling=df.min_rolling.shift(1))

In [ ]:
df = df.assign(last_low=df.low.shift(1))
df = df.assign(next_low=df.low.shift(-1))

In [ ]:
df = df.assign(local_low=((df.low < df.last_low) & (df.low < df.next_low)))

In [ ]:
df = df.assign(pivot=((df.min_rolling != df.rolling) & df.local_low))

In [ ]:
chart_df = df.copy()

In [ ]:
import numpy as np

def extract_pivots(df):
    index = (df.min_rolling != df.last_min_rolling) & df.local_low
    return df.loc[index]


def add_trend_lines(df):
    pivots = extract_pivots(df)
    pivots = pivots.assign(last_date=pivots.date.shift(1))
    pivots = pivots.assign(last_low=pivots.low.shift(1))
    run = (pivots.date - pivots.last_date).dt.total_seconds() / 3600.0
    rise = pivots.low - pivots.last_low
    pivots = pivots.assign(slope=rise.astype(float) / run)
    pivots = pivots.assign(slope=pivots.slope.fillna(method='bfill'))
    df = df.assign(trend=np.nan, back_trend=np.nan, slope=np.nan, back_slope=np.nan)
    df.loc[pivots.index, 'slope'] = pivots.slope
    df = df.assign(slope=df.slope.fillna(method='ffill'))
    df.loc[pivots.index, 'back_slope'] = df.loc[pivots.index, 'slope']
    df = df.assign(back_slope=df.back_slope.fillna(method='bfill'))
    started = False
    back_started = False
    line_index = 0
    back_line_index = 0
    for index in range(1, df.shape[0]):
        if ((df.iloc[index - 1].slope != df.iloc[index].slope) and not np.isnan(df.iloc[index].slope) and not started):
            line_index = 0
            started = True
        elif (df.iloc[index - 1].slope != df.iloc[index].slope):
            line_index = 0
        if started:
            location = df.index[index]
            last_location = df.index[index - 1]
            if df.iloc[index].slope > 0:
                df.loc[location, 'trend'] = df.loc[location, "low"] if line_index == 0 else df.loc[last_location, "trend"] + df.iloc[index].slope
            line_index += 1
        end_index = df.shape[0] - 1 - index
        last_end_index = df.shape[0] - index
        # extract this into function
        if ((df.iloc[end_index].back_slope != df.iloc[last_end_index].back_slope) and not np.isnan(df.iloc[end_index].back_slope) and not back_started):
            back_line_index = 0
            back_started = True
        elif (df.iloc[end_index].back_slope != df.iloc[last_end_index].back_slope):
            back_line_index = 0
        if back_started:
            location = df.index[end_index]
            last_location = df.index[last_end_index]
            if df.iloc[end_index].back_slope > 0:
                df.loc[location, 'back_trend'] = df.loc[location, "low"] if back_line_index == 0 else df.loc[last_location, "back_trend"] - df.iloc[end_index].back_slope
            back_line_index += 1
            
        
    return df, pivots

In [ ]:
trend_df, pivs = add_trend_lines(df)

In [ ]:
import plotly.graph_objs as go


def plot_data(df, pivots):
    fig = go.Figure()
    fig.add_trace(
        go.Candlestick(
        open=df.open, high=df.high, low=df.low,
        close=df.close, x=df.date
        )  
    )
    '''
    fig.add_trace(
        go.Scatter(
            x = [pd.Timestamp('2022-07-26 08:00:00'), pd.Timestamp('2022-07-26 17:15:00')],
            y = [1.19635, 1.1986], mode='lines', marker=dict(color='blue')
        )
    )
    '''
    # pivots_df = extract_pivots(df)
    
    fig.add_trace(
        go.Scatter(
            x=df.date, y=df.trend, mode='markers', marker=dict(color='blue'),
            name='trend'
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.date, y=df.back_trend, mode='markers', marker=dict(color='purple'),
            name='back trend'
        )
    )
    
    pivots_df = extract_pivots(df)
    fig.add_trace(
        go.Scatter(
            x=pivots.date, y=pivots.low, mode='markers', marker=dict(color='black'),
            name='pivots'
        )
    )
    
    fig.update_layout(xaxis_rangeslider_visible=False, height=700)
    return fig

In [ ]:
fig = plot_data(trend_df, pivs)

In [ ]:
fig